# First Test for Volatility Based VaR

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model
from itertools import product

# Import functions
import VolatilityVaR
import Backtesting
import InteractivePlots
import ExpectedShortfall as ES

### Data

In [2]:
# Download data
sp500_data = yf.download("^GSPC", start="2017-01-01", end="2021-01-01")
sp500_data["Log Returns"] = np.log(sp500_data["Close"] / sp500_data["Close"].shift(1))
returns = sp500_data["Log Returns"].dropna()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


### Garch based VaR on S&P 500

In [3]:
# Set parameters
confidence_level = 0.99 # <----- Can choose 0.95 etc

In [4]:
# Apply GARCH volatility model
garch_output, next_day_var = VolatilityVaR.var_garch(returns, confidence_level, vol_model="GJR")

print(f"Next-day GARCH VaR estimate (abs): {next_day_var:.2f}%")

# Backtest full period
violations, rate = Backtesting.backtest_var(garch_output, confidence_level)
print(f"Full Period — Violations: {violations}, Rate: {rate:.2f}%")

# Backtest a subset
viol_subset, rate_subset = Backtesting.subset_backtest_var(garch_output, "2019-11-01", "2020-11-30")
print(f"Subset — Violations: {viol_subset}, Rate: {rate_subset:.2f}%")

# Compute ES for the whole period
garch_output = ES.compute_expected_shortfall_volatility(garch_output, confidence_level)

# Plot interactive VaR
fig_var = InteractivePlots.interactive_plot_var(garch_output)
fig_var.show()

# Plot interactive ES for a subset
fig_es = InteractivePlots.interactive_plot_es(garch_output, subset=("2019-11-01", "2020-11-30"))
fig_es.show()

# Plot interactive volatility for a subset
fig_vol = InteractivePlots.interactive_plot_volatility(garch_output["Volatility"], subset=("2019-11-01", "2020-11-30"))
fig_vol.show()


Next-day GARCH VaR estimate (abs): 1.45%
Full Period — Violations: 11, Rate: 1.09%
Subset — Violations: 4, Rate: 1.47%


### Garch Forecasts

In [5]:
# Configuration grid
steps_list = [1, 5, 10]
cumulative_list = [False, True]
distributions = ["normal", "t", "ged"]

print("GARCH(1,1) VaR Forecasts\n" + "-" * 30)

for steps_ahead, cumulative, dist in product(steps_list, cumulative_list, distributions):
    try:
        var_value = VolatilityVaR.garch_forecast(
            returns=returns,
            steps_ahead=steps_ahead,
            cumulative=cumulative,
            compute_var=True,
            confidence_level=0.99,
            distribution=dist
        )

        label = f"VaR | {steps_ahead}-day | {'CUM' if cumulative else 'Single'} | {dist.upper()}"
        print(f"{label:<45}: {var_value:.4f}%")  # Removed extra % formatting

    except Exception as e:
        print(f"{label:<45}: ERROR → {e}")

# Variances (print volatility in percentage)
print("\nGARCH(1,1) Variance Forecasts")
print("-------------------------------")

for horizon in [1, 5, 10]:
    for cumulative in [False, True]:
        var_type = "CUM" if cumulative else "Single"
        variance = VolatilityVaR.garch_forecast(
            returns,
            steps_ahead=horizon,
            cumulative=cumulative,
            compute_var=False
        )
        std_dev = np.sqrt(variance)
        print(f"STD | {horizon}-day | {var_type:6} | {std_dev * 100:.4f}%")


GARCH(1,1) VaR Forecasts
------------------------------
VaR | 1-day | Single | NORMAL                : 1.3652%
VaR | 1-day | Single | T                     : 1.5218%
VaR | 1-day | Single | GED                   : 1.5279%
VaR | 1-day | CUM | NORMAL                   : 1.3098%
VaR | 1-day | CUM | T                        : 1.4601%
VaR | 1-day | CUM | GED                      : 1.4660%
VaR | 5-day | Single | NORMAL                : 1.5578%
VaR | 5-day | Single | T                     : 1.7365%
VaR | 5-day | Single | GED                   : 1.7435%
VaR | 5-day | CUM | NORMAL                   : 3.1669%
VaR | 5-day | CUM | T                        : 3.5302%
VaR | 5-day | CUM | GED                      : 3.5445%
VaR | 10-day | Single | NORMAL               : 1.7520%
VaR | 10-day | Single | T                    : 1.9530%
VaR | 10-day | Single | GED                  : 1.9609%
VaR | 10-day | CUM | NORMAL                  : 4.8450%
VaR | 10-day | CUM | T                       : 5.4009%
VaR | 10-

### MA based VaR on the same data

In [6]:
# Apply MA volatility model
ma_output, next_day_var = VolatilityVaR.var_moving_average(returns, confidence_level)

print(f"Next-day MA VaR estimate (abs): {next_day_var:.2f}%")

# Backtest full period
violations, rate = Backtesting.backtest_var(ma_output, confidence_level)
print(f"Full Period — Violations: {violations}, Rate: {rate:.2f}%")

# Backtest a subset
viol_subset, rate_subset = Backtesting.subset_backtest_var(ma_output, "2019-11-01", "2020-11-30")
print(f"Subset — Violations: {viol_subset}, Rate: {rate_subset:.2f}%")

# Compute ES for the whole period
ma_output = ES.compute_expected_shortfall_volatility(ma_output, confidence_level)

# Plot interactive VaR
fig_var = InteractivePlots.interactive_plot_var(ma_output)
fig_var.show()

# Plot interactive ES for a subset
fig_es = InteractivePlots.interactive_plot_es(ma_output, subset=("2019-11-01", "2020-11-30"))
fig_es.show()

# Plot interactive volatility for a subset
fig_vol = InteractivePlots.interactive_plot_volatility(ma_output["Volatility"], subset=("2019-11-01", "2020-11-30"))
fig_vol.show()


Next-day MA VaR estimate (abs): 1.48%
Full Period — Violations: 10, Rate: 1.01%
Subset — Violations: 4, Rate: 1.47%


### Appendix

#### 1. GARCH(p, q) VaR  
Standard GARCH captures volatility clustering using past squared shocks and variances.

$$
r_t = \mu + \varepsilon_t, \quad \varepsilon_t = \sigma_t z_t
$$

$$
\sigma_t^2 = \omega + \sum_{i=1}^q \alpha_i \varepsilon_{t-i}^2 + \sum_{j=1}^p \beta_j \sigma_{t-j}^2
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 2. EGARCH(p, q) VaR  
EGARCH models log-volatility and captures asymmetry in the volatility response.

$$
\log(\sigma_t^2) = \omega + \sum_{i=1}^q \beta_i \log(\sigma_{t-i}^2) + \sum_{j=1}^p \alpha_j \left( \frac{|\varepsilon_{t-j}|}{\sigma_{t-j}} - \mathbb{E}\left[ \frac{|\varepsilon_{t-j}|}{\sigma_{t-j}} \right] \right) + \gamma_j \frac{\varepsilon_{t-j}}{\sigma_{t-j}}
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 3. GJR-GARCH(p, q) VaR  
GJR-GARCH adds an indicator term to account for the leverage effect of negative shocks.

$$
\sigma_t^2 = \omega + \sum_{i=1}^q \alpha_i \varepsilon_{t-i}^2 + \sum_{j=1}^p \beta_j \sigma_{t-j}^2 + \sum_{i=1}^q \gamma_i \varepsilon_{t-i}^2 \cdot \mathbb{I}_{\{\varepsilon_{t-i}<0\}}
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 4. APARCH(p, q) VaR  
APARCH generalizes GARCH with power terms and asymmetry.

$$
\sigma_t^\delta = \omega + \sum_{i=1}^q \alpha_i \left( |\varepsilon_{t-i}| - \gamma_i \varepsilon_{t-i} \right)^\delta + \sum_{j=1}^p \beta_j \sigma_{t-j}^\delta
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 5. ARCH(p) VaR  
ARCH models volatility using only past squared residuals.

$$
\sigma_t^2 = \omega + \sum_{i=1}^p \alpha_i \varepsilon_{t-i}^2
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 6. EWMA VaR  
EWMA assigns exponentially decaying weights to past squared returns.

$$
\sigma_t^2 = \lambda \sigma_{t-1}^2 + (1 - \lambda) r_{t-1}^2
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 7. MA VaR  
Moving Average volatility is estimated using a rolling window of past squared returns.

$$
\sigma_t = \sqrt{ \frac{1}{n} \sum_{i=1}^n r_{t-i}^2 }
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 8. Expected Shortfall (ES)  
Expected Shortfall estimates the conditional expectation of loss beyond the VaR.

$$
\text{ES}_t = -\hat{\sigma}_t \cdot \mathbb{E}[z_t \mid z_t < z_\alpha]
$$

---

#### 9. GARCH(1,1) Variance Forecast – Analytical  

Forecast for single-step variance $T$ steps ahead:

$$
\mathbb{E}[\sigma_{t+T}^2] = \text{VL} + (\alpha + \beta)^T \cdot (\sigma_t^2 - \text{VL})
$$

Forecast for cumulative variance over $T$ days:

$$
\mathbb{E}[\sigma_{t,T}^2] =
\text{VL} \cdot \left( T - 1 - \frac{(\alpha + \beta)(1 - (\alpha + \beta)^{T - 1})}{1 - (\alpha + \beta)} \right)
+ \sigma_t^2 \cdot \frac{1 - (\alpha + \beta)^T}{1 - (\alpha + \beta)}
$$

Where:

- $\text{VL} = \dfrac{\omega}{1 - \alpha - \beta}$ is the long-run variance  
- $\sigma_t^2$ is the last fitted conditional variance  


#### 10. GARCH(1,1) VaR Forecast – Analytical  

Once variance is forecasted, the VaR over the horizon $T$ is:

$$
\text{VaR}_{t,T} = - z_\alpha \cdot \sqrt{\mathbb{E}[\sigma_{t,T}^2]}
$$

Where $z_\alpha$ is the quantile of the fitted standardized innovation distribution:

- Normal: $z_\alpha = \Phi^{-1}(1 - \alpha)$  
- Student-t / GED: fitted on standardized residuals
